In [1]:
%reload_ext rpy2.ipython
%reload_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

In [56]:
%%R

require('ggplot2')

In [2]:
%%R

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Lade nötiges Paket: tidyverse


In [43]:
%%R

total_tiles <- 100  

# Normalize count values to fit into the grid
df_grid <- df %>%
  mutate(tiles = round(Count / sum(Count) * total_tiles)) %>%
  arrange(desc(tiles))

# Expand the data into individual tiles
df_expanded <- df_grid %>%
  uncount(tiles) %>%
  mutate(tile_id = row_number())

# Generate x, y positions for a grid layout
df_expanded <- df_expanded %>%
  mutate(
    x = (tile_id - 1) %% 10,  # 10 columns
    y = (tile_id - 1) %/% 10   # Row index
  )

In [48]:
%%R

chart = ggplot(df_expanded, aes(x = x, y = -y, fill = `Product Type`)) +
  geom_tile(color = "white", width = 0.9, height = 0.9) +  # Keep square tiles
  scale_fill_brewer(palette = "Set2") +  
  theme_minimal() +
  labs(
    title = "FDA Recalls by Product Type (2024)",
    subtitle = "Each tile represents 10 recall cases",
    fill = "Product Type"
  ) +
  coord_fixed(ratio = 1) +  # Ensures tiles remain squares
  theme(
    axis.text = element_blank(),
    axis.ticks = element_blank(),
    panel.grid = element_blank()
  )


In [60]:
%%R

ggsave('tile.svg', plot = chart, width = 8, height = 6)